In [1]:
# Install the Antigranular package
!pip install antigranular &> /dev/null

In [1]:
import antigranular as ag
session = ag.login(<client_id>,<client_secret>, competition = "Global Economic Forecast Hackathon with Texas A&M Aggie Data Science")

Dataset "Statistical Performance Indicators" loaded to the kernel as statistical_performance_indicators
Key Name                       Value Type     
---------------------------------------------
train_x                        PrivateDataFrame
train_y                        PrivateDataFrame
test_x                         DataFrame      

Connected to Antigranular server session id: c7933b1a-6909-4db2-bd51-e2eac1dba575, the session will time out if idle for 25 minutes
Cell magic '%%ag' registered successfully, use `%%ag` in a notebook cell to execute your python code on Antigranular private python server
🚀 Everything's set up and ready to roll!


In [3]:
%%ag
x_train = statistical_performance_indicators["train_x"]
y_train = statistical_performance_indicators["train_y"]
x_test = statistical_performance_indicators["test_x"]

In [380]:
%%ag
ag_print(x_train.info())

+----+-----------------+-------------+---------------+---------+----------+
|    | Column          | numerical   | categorical   | dtype   | bounds   |
|----+-----------------+-------------+---------------+---------+----------|
|  0 | intl_org_use    | True        | False         | float64 | (1, 3)   |
|  1 | social_stats    | True        | False         | float64 | (0, 1)   |
|  2 | economic_stats  | True        | False         | float64 | (0, 1)   |
|  3 | inst_stats      | True        | False         | float64 | (0, 1)   |
|  4 | pov_ratio       | True        | False         | float64 | (0, 1)   |
|  5 | child_mortality | True        | False         | float64 | (0, 1)   |
|  6 | debt_service    | True        | False         | float64 | (0, 1)   |
|  7 | safe_water      | True        | False         | float64 | (0, 1)   |
|  8 | labor_force     | True        | False         | float64 | (0, 1)   |
|  9 | no_poverty      | True        | False         | float64 | (0, 1)   |
| 10 | zero_

In [391]:
%%ag
def feat_eng(df):
    df['sdg_social'] = (df['no_poverty'] + df['zero_hunger'] + df['good_health'] + df['quality_edu'] + df['gender_eq']) / 5
    df['sdg_environmental'] = (df['clean_water'] + df['clean_energy'] + df['life_land']) / 3
    df['sdg_economic'] = (df['decent_work'] + df['innovation'] + df['reduced_ineq']) / 3
    df['sdg_governance'] = (df['cities'] + df['peace_justice'] + df['partnerships']) / 3
    
    # 2. Statistical Quality Index
    df['stats_quality'] = (df['social_stats'] + df['economic_stats'] + df['inst_stats']) / 3
    
    # 3. Data Coverage Ratio
    df['data_coverage'] = (df['pov_ratio'] + df['child_mortality'] + df['debt_service'] + df['safe_water'] + df['labor_force']) / 5
    
    # 4. Interaction Features
    df['org_stats_interaction'] = df['intl_org_use'] * df['stats_quality']
    df['poverty_health_interaction'] = df['pov_ratio'] * df['child_mortality']
    
    # 5. Squared Terms
    df['intl_org_use_squared'] = df['intl_org_use'] ** 2
    df['stats_quality_squared'] = df['stats_quality'] ** 2
    
    # 6. Ratios
    df['social_economic_ratio'] = df['sdg_social'] / (df['sdg_economic']+0.0000001)
    df['environmental_governance_ratio'] = df['sdg_environmental'] / (df['sdg_governance']+0.0000001)
    
    # 7. Composite Scores
    df['overall_sdg_score'] = (df['sdg_social'] + df['sdg_environmental'] + df['sdg_economic'] + df['sdg_governance']) / 4
    
    # 8. Statistical Quality vs Implementation Gap
    df['stats_implementation_gap'] = df['stats_quality'] - df['overall_sdg_score']
    return df
x_train = feat_eng(x_train)
x_test = feat_eng(x_test)

In [392]:
%%ag
# x_train['social_indicators'] = (x_train['no_poverty'] + x_train['zero_hunger'] + 
#                                x_train['good_health'] + x_train['quality_edu'] + 
#                                x_train['gender_eq']) / 5
# x_test['social_indicators'] = (x_test['no_poverty'] + x_test['zero_hunger'] + 
#                               x_test['good_health'] + x_test['quality_edu'] + 
#                               x_test['gender_eq']) / 5

# 2. Çevre ve Enerji (3 gösterge)
x_train['environment_energy'] = (x_train['clean_water'] + x_train['clean_energy'] + 
                                x_train['life_land']) / 3
x_test['environment_energy'] = (x_test['clean_water'] + x_test['clean_energy'] + 
                               x_test['life_land']) / 3

# 3. Ekonomik Göstergeler (3 gösterge)
x_train['economic_indicators'] = (x_train['decent_work'] + x_train['innovation'] + 
                                 x_train['reduced_ineq']) / 3
x_test['economic_indicators'] = (x_test['decent_work'] + x_test['innovation'] + 
                                x_test['reduced_ineq']) / 3

# 4. Kurumsal ve Yönetişim (3 gösterge)
x_train['institutional_governance'] = (x_train['cities'] + x_train['peace_justice'] + 
                                     x_train['partnerships']) / 3
x_test['institutional_governance'] = (x_test['cities'] + x_test['peace_justice'] + 
                                    x_test['partnerships']) / 3

# 5. İstatistiksel Metrikler (3 gösterge)
x_train['statistical_metrics'] = (x_train['social_stats'] + x_train['economic_stats'] + 
                                 x_train['inst_stats']) / 3
x_test['statistical_metrics'] = (x_test['social_stats'] + x_test['economic_stats'] + 
                                x_test['inst_stats']) / 3

# 6. Temel İhtiyaç Göstergeleri (3 gösterge)
x_train['basic_needs'] = (x_train['pov_ratio'] + x_train['child_mortality'] + 
                         x_train['safe_water']) / 3
x_test['basic_needs'] = (x_test['pov_ratio'] + x_test['child_mortality'] + 
                        x_test['safe_water']) / 3

# 7. Finansal ve İş Göstergeleri (2 gösterge)
x_train['financial_labor'] = (x_train['debt_service'] + x_train['labor_force']) / 2
x_test['financial_labor'] = (x_test['debt_service'] + x_test['labor_force']) / 2

# # 8. Toplam SDG Skoru (15 gösterge)
# x_train['total_sdg_score'] = (x_train['no_poverty'] + x_train['zero_hunger'] + 
#                              x_train['good_health'] + x_train['quality_edu'] + 
#                              x_train['gender_eq'] + x_train['clean_water'] + 
#                              x_train['clean_energy'] + x_train['decent_work'] + 
#                              x_train['innovation'] + x_train['reduced_ineq'] + 
#                              x_train['cities'] + x_train['consump_prod'] + 
#                              x_train['life_land'] + x_train['peace_justice'] + 
#                              x_train['partnerships']) / 15
# x_test['total_sdg_score'] = (x_test['no_poverty'] + x_test['zero_hunger'] + 
#                             x_test['good_health'] + x_test['quality_edu'] + 
#                             x_test['gender_eq'] + x_test['clean_water'] + 
#                             x_test['clean_energy'] + x_test['decent_work'] + 
#                             x_test['innovation'] + x_test['reduced_ineq'] + 
#                             x_test['cities'] + x_test['consump_prod'] + 
#                             x_test['life_land'] + x_test['peace_justice'] + 
#                             x_test['partnerships']) / 15

In [53]:
%%ag
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from op_tensorflow import PrivateKerasModel, PrivateDataLoader
import op_tensorflow
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from op_tensorflow import PrivateKerasModel, PrivateDataLoader
import op_pandas
from op_pandas import standard_scaler, PrivateDataFrame, train_test_split


concat_df = x_train.join(y_train)
train_data, test_data = op_pandas.train_test_split(concat_df, test_size=0.0001, random_state=42)


train_y = train_data['income'].copy()
train_data.drop(columns=['income'], inplace=True)

test_y = test_data['income'].copy()
test_data.drop(columns=['income'], inplace=True)

batch_size = 32
epochs = 15
noise_multiplier = 0.3
target_delta = 1

# Model mimarisi - 20 epoch için optimize edilmiş
model = Sequential([
    Conv1D(256, 3, activation='relu', input_shape=(24, 1)),
    BatchNormalization(),
    Dropout(0.2),
    Conv1D(128, 3, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.2),
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])


# DP model
dp_model = PrivateKerasModel(
    model=model,
    l2_norm_clip=1,  # Biraz daha yüksek
    noise_multiplier=noise_multiplier
)



optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

# Model compile
dp_model.compile(
    optimizer=optimizer,
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Data loader
data_loader = PrivateDataLoader(
    feature_df=train_data,
    label_df=train_y,
    batch_size=batch_size
)

# Model eğitimi - callbacks olmadan
dp_model.fit(
    x=data_loader,
    epochs=epochs,
    target_delta=target_delta,
    #validation_data=(test_data, test_y)
)




Epoch 1/15

115/115 - 30s - loss: 0.4896 - accuracy: 0.7513 - 30s/epoch - 263ms/step

Epoch 2/15

115/115 - 21s - loss: 0.4302 - accuracy: 0.7867 - 21s/epoch - 186ms/step

Epoch 3/15

115/115 - 21s - loss: 0.4124 - accuracy: 0.7883 - 21s/epoch - 185ms/step

Epoch 4/15

115/115 - 21s - loss: 0.3942 - accuracy: 0.8037 - 21s/epoch - 186ms/step

Epoch 5/15

115/115 - 22s - loss: 0.3804 - accuracy: 0.8164 - 22s/epoch - 189ms/step

Epoch 6/15

115/115 - 22s - loss: 0.3830 - accuracy: 0.8166 - 22s/epoch - 188ms/step

Epoch 7/15

115/115 - 21s - loss: 0.3548 - accuracy: 0.8284 - 21s/epoch - 185ms/step

Epoch 8/15

115/115 - 21s - loss: 0.3443 - accuracy: 0.8316 - 21s/epoch - 184ms/step

Epoch 9/15

115/115 - 21s - loss: 0.3329 - accuracy: 0.8454 - 21s/epoch - 182ms/step

Epoch 10/15

115/115 - 21s - loss: 0.3128 - accuracy: 0.8543 - 21s/epoch - 187ms/step

Epoch 11/15

115/115 - 21s - loss: 0.3067 - accuracy: 0.8578 - 21s/epoch - 185ms/step

Epoch 12/15

115/115 - 21s - loss: 0.2927 - accuracy

In [58]:
%%ag
from op_pandas import standard_scaler, PrivateDataFrame
test=PrivateDataFrame(x_test).copy()
y_pred = dp_model.predict(test, label_columns=["output"])
# Note that the predictions are a float scalar
# so we scale it
def f(x: float) -> int:
  if x > 0.5:
    return 1
  else:
    return 0

y_pred["output"] = y_pred["output"].map(f, output_categories=[0, 1])
result = submit_predictions(y_pred)

13/13 [==============================] - 1s 34ms/step

score: {'leaderboard': 0.9318408259176966, 'logs': {'BIN_ACC': 0.9318408259176966, 'LIN_EPS': -0.0}}

